In [1]:
import numpy as np
from matplotlib import pyplot as plt

# Simulation results of the three approaches

The simulation results of solving Max-Cut on $100$ Erdos-Renyi graphs are listed in `sim-results.json`.
For each entry, we give the random seed corresponding to that graph, and the approximation ratio achieved by three methods, among which `cp`, `qiq` and `gw` stands for our method, QAOA-in-QAOA and Goemans-Williamson respectively.

The following cell caintains the code used to generate the plot comparing three methods.

In [10]:
import json

from rsmf import CustomFormatter

formatter = CustomFormatter(
    columnwidth=542.02501 * 0.01389,
    wide_columnwidth=542.02501 * 0.01389,
    fontsizes=12,
    pgf_preamble=r"""\usepackage{mathptmx}
    \usepackage{newtxtext,newtxmath}
    \renewcommand{\familydefault}{\sfdefault}""",
)

plt.rcParams["font.family"] = "serif"
plt.rcParams["text.usetex"] = True

formatter.figure(aspect_ratio=0.5)

data = json.load(open("data/sim-results.json"))

plt.cla()

cq, qiq, gw = np.zeros(len(data)), np.zeros(len(data)), np.zeros(len(data))
for item in data:
    cq[item["seed"]] = item["cq"]
    qiq[item["seed"]] = item["qiq"]
    gw[item["seed"]] = item["gw"]


LINE_WIDTH, MARKER_SIZE = 0.1, 2
print(
    f"on {sum(1 if cq[i] > max(gw[i], qiq[i]) else 0 for i in range(len(data)))}/{len(data)} instances cq outperforms the other 2"
)
plt.plot(
    np.arange(len(data)),
    cq,
    "s--",
    label="QAOA coupled local search",
    linewidth=LINE_WIDTH,
    markersize=MARKER_SIZE,
)
plt.plot(
    np.arange(len(data)),
    qiq,
    "o--",
    label="QAOA in QAOA",
    linewidth=LINE_WIDTH,
    markersize=MARKER_SIZE,
)
plt.plot(
    np.arange(len(data)),
    gw,
    "x--",
    label="Goemans-Williamson",
    linewidth=LINE_WIDTH,
    markersize=MARKER_SIZE,
)

L, R = -5, 105
ALPHA = 1

AVG_LW = 0.5

avg_ratios = [np.average(cq), np.average(qiq), np.average(gw)]
stds = [np.std(cq), np.std(qiq), np.std(gw)]
print("average =", " ".join([f"%.4f" % x for x in avg_ratios]))
print("std deviation =", " ".join([f"{x:.3e}" for x in stds]))

xlim = plt.xlim()
for i in range(len(avg_ratios)):
    plt.plot(
        [0, 100],
        [avg_ratios[i], avg_ratios[i]],
        color=f"C{i}",
        linewidth=AVG_LW,
        alpha=ALPHA,
    )
    plt.plot(
        [-5, 0],
        [avg_ratios[i], avg_ratios[i]],
        "--",
        color=f"C{i}",
        linewidth=AVG_LW,
        alpha=ALPHA,
    )

    plt.fill_between(
        [-5, 200],
        avg_ratios[i] - stds[i],
        avg_ratios[i] + stds[i],
        color=f"C{i}",
        alpha=0.1,
        edgecolor="none",
    )

ticks = np.concatenate((np.arange(0.93, 1.1, 0.01), avg_ratios))
ticks.sort()

labels = []
for x in ticks:
    if x in avg_ratios:
        labels.append("%.4f" % (x))
    else:
        labels.append("%.2f" % (x))

_, label_objs = plt.yticks(ticks, labels)

for i in range(len(ticks)):
    if ticks[i] in avg_ratios:
        label_objs[i].set_fontsize("small")
        label_objs[i].set_color(f"C{avg_ratios.index(ticks[i])}")

plt.xticks(np.arange(0, 110, 10))
plt.xlim(L, R)
plt.ylim(0.925, 1.005)

plt.xlabel("random seed")
plt.ylabel("approximation ratio")
plt.legend()

plt.tight_layout()
plt.savefig("fig/cmp.pdf", bbox_inches="tight", pad_inches=0)

on 96/100 instances cq outperforms the other 2
average = 0.9950 0.9778 0.9643
std deviation = 2.627e-03 9.575e-03 8.413e-03


# Demonstrating the increment of $|T_{\rm g}|$
The increment of $|T_{\rm g}|$ for different settings are listed in `T_greedy-sizes.json`. For each setting, we generate $20$ random graphs, and for each graph, we generate $20$ permutations of $\{0,1,\dots, 2^{n_0}-1\}$, and compute $T_{\rm g}$ accordingly.

Results for different graphs with the same setting are averaged and plotted in the following cell.

In [3]:
data = json.load(open("data/T_greedy-sizes.json"))


def plot_with_shade(data, small=False):
    plt.cla()

    if small:
        ax = formatter.figure(width_ratio=0.3, aspect_ratio=0.7).gca()
    else:
        ax = formatter.figure(width_ratio=0.6).gca()

    ax.yaxis.get_major_locator().set_params(integer=True)

    mn = np.concatenate(([1], np.min(data, axis=(0, 1))))
    mx = np.concatenate(([1], np.max(data, axis=(0, 1))))
    avg = np.concatenate(([1], np.average(data, axis=(0, 1))))

    xs = list(range(len(avg)))
    plt.plot(xs, avg)
    plt.fill_between(xs, mn, mx, alpha=0.1)

    if not small:
        plt.xlabel("$|U_0|$")
        plt.ylabel("$|T_{\\rm g}(U_0)|$")
        plt.plot([0, 27], [1 + 0.1, avg[-1] + 0.1], "--", color="grey")

    ticks = plt.xticks()[0][1:-1]
    plt.xticks(ticks, ["$2^{%d}$" % (x) for x in ticks])
    plt.tight_layout()


def sig_tuple(x):
    return "-".join(
        map(
            str,
            [
                x["graph_size"],
                x["split_ratio"],
                x["beta_guarantee"],
                x["connectivity"],
            ],
        )
    )


def get_matrix(x):
    return np.array([x["graphs"][i]["perm_results"] for i in range(len(x["graphs"]))])


plot_with_shade(get_matrix(data[0]), small=False)
plt.savefig(f"fig/{sig_tuple(data[0])}-avg.pdf", bbox_inches="tight", pad_inches=0)

for i in range(1, len(data)):
    plot_with_shade(get_matrix(data[i]), small=True)
    plt.savefig(f"fig/{sig_tuple(data[i])}-avg.pdf", bbox_inches="tight", pad_inches=0)

for seed in range(2):
    plot_with_shade(np.array([data[0]["graphs"][seed]["perm_results"]]), small=True)
    plt.savefig(
        f"fig/{sig_tuple(data[0])}-seed{seed}.pdf", bbox_inches="tight", pad_inches=0
    )